# **Tarea Nº 4**

- Fecha y hora de entrega: Domingo 12 de enero, 11:59pm
- Agregue los nombres de las personas con las que discutió esta tarea: ____ Armando Paredes ______
- Envíe su tarea haciendo el `push` de su código a su repo en GitHub Classroom: 

## **Mercado de autos usados**

Importa el conjunto de datos `neoauto_20240924.csv`
    

1. ¿Cuál es el precio promedio de los autos de cada marca (`item_brand`) publicados en cada subcategoría (`item_category_2`)?

In [ ]:
# Solución

2. ¿Cuántos autos tienen más de 100,000 kilómetros (`item_km`) pero un precio (`item_price`) menor al promedio general?

In [ ]:
# Solución

3. Encuentra el modelo (`item_name`) más caro y el más barato de cada marca (`item_brand`).

In [ ]:
# Solución

4. Encuentra las tres marcas con mayor número de autos financiados por Santander (`item_financed_by`). 

In [ ]:
# Solución

5. Agrupa las publicaciones por tipo de transmisión (`item_transmission`) y calcula la desviación estándar del precio (`item_price`) para cada tipo.  

In [ ]:
# Solución

6. ¿Cuál es el kilometraje promedio de las publicaciones con el tag "Premium" y de las publicaciones con el tag "Como nuevo"?

In [ ]:
# Solución

7. ¿Cuáles son las marcas de vehículas para los que todos sus modelos fabricados en los últimos 5 años tienen al menos 5 publicaciones cada una?

In [ ]:
# Solución

## **Evolución económica internacional**

Importa el conjunto de datos de la hoja "Data" del archivo `pwt1001.xlsx`. El diccionario de variables está disponible en la hoja "Legend" del archivo Excel.
    

1. Calcule el PBI per cápita real ajustado por poder de paridad de compra (PPP). Use la variable de PBI que se calcula por gasto y la población. ¿Cuáles son los 10 países top?

In [ ]:
# 1. 
# Cargar los datos
file_path = "C:/Users/HP/Downloads/pwt1001.xlsx"
data = pd.read_excel(file_path, sheet_name="Data")
# Calcular el PBI per cápita real PPP
data['rgdpe_pc'] = data['rgdpe'] / data['pop']
# Eliminar valores faltantes
valid_data = data.dropna(subset=['rgdpe_pc', 'year'])
# Obtener el año más reciente
latest_year = valid_data['year'].max()
# Identificar los top 10 países
top_countries = (valid_data[valid_data['year'] == latest_year]
                .nlargest(10, 'rgdpe_pc')
                [['country', 'year', 'rgdpe_pc']]
                .reset_index(drop=True))

print(f"\nTop 10 países por PBI per cápita PPP ajustado (Año {int(latest_year)}):")
print("\nRanking  País                  PBI per cápita PPP")
print("-" * 50)
for idx, row in top_countries.iterrows():
    print(f"{idx+1:<8} {row['country']:<20} {row['rgdpe_pc']:,.2f}")

2. Genera el ránking de los países con el mayor incremento del PBI per cápita real PPP entre los años 1990 y 2019. ¿Dónde se ubica Perú? Para los países que no tengan información para dichos años, puede utilizar datos provenientes de hasta dos años antes o después

In [ ]:
# 2.
data_1990_2019 = valid_data[(valid_data['year'] >= 1988) & (valid_data['year'] <= 2021)]
df_changes = data_1990_2019.pivot(index='country', columns='year', values='rgdpe_pc')
df_changes['change'] = df_changes[2019] - df_changes[1990]
ranking_changes = df_changes.sort_values('change', ascending=False).reset_index()
ranking_peru = ranking_changes[ranking_changes['country'] == 'Peru'].index[0] + 1

print("Ranking de países con mayor incremento (1990-2019):")
print(ranking_changes[['country', 'change']].head(10))
print(f"\nUbicación de Perú: {ranking_peru}° lugar")

3. Para el periodo 2010-2019, calcule el promedio del índice del capital humano. ¿Qué relación existe entre esta variable y el PBI per cápita real PPP calculado previamente? ¿y el número de horas laboradas al año por los trabajadores?

In [ ]:
# 3.
data_2010_2019 = valid_data[(valid_data['year'] >= 2010) & (valid_data['year'] <= 2019)]
human_capital_avg = data_2010_2019.groupby('country')['hc'].mean().reset_index()
relation_df = data_2010_2019.groupby('country').agg({
    'rgdpe_pc': 'mean',
    'hc': 'mean',
    'avh': 'mean'
}).reset_index()
correlacion_hc_gdp = relation_df['hc'].corr(relation_df['rgdpe_pc'])
correlacion_hc_avh = relation_df['hc'].corr(relation_df['avh'])
print("Promedio del índice de capital humano por país (2010-2019):")
print(human_capital_avg.sort_values('hc', ascending=False).head(10))
print("\nCorrelaciones:")
print(f"Capital Humano - PBI per cápita: {correlacion_hc_gdp:.3f}")
print(f"Capital Humano - Horas trabajadas: {correlacion_hc_avh:.3f}")

4. A partir de la variable del precio de los hogares, calcula los países y los años en los cuales se registró la mayor inflación anual para todo el periodo de tiempo disponible en la base de datos

In [ ]:
# 4.
data_sorted = data.sort_values(['country', 'year'])
data_sorted['inflation'] = data_sorted.groupby('country')['pl_con'].pct_change() * 100
data_sorted = data_sorted.replace([float('inf'), float('-inf')], float('nan'))
max_inflation = data_sorted.dropna(subset=['inflation'])
top_inflation = (max_inflation
                .sort_values('inflation', ascending=False)
                .head(10)[['country', 'year', 'inflation']]
                .reset_index(drop=True))
print("Top 10 casos de mayor inflación anual registrada:")
print("\nPaís\t\tAño\tInflación (%)")
print("-" * 40)
for _, row in top_inflation.iterrows():
    print(f"{row['country']:<15} {int(row['year'])}\t{row['inflation']:,.1f}%")

5. ¿Cuáles son los países con la mayor cantidad de missing values en la variable de productividad total de factores a precios corrientes?

In [ ]:
# 5.
missing_tfp = (data
               .groupby('country')['ctfp']  # Agrupamos por país y usamos la variable ctfp
               .apply(lambda x: x.isna().sum())  # Contamos NaN
               .sort_values(ascending=False)  # Ordenamos de mayor a menor
               .reset_index(name='missing_count'))  # Reiniciamos el índice
total_years = data.groupby('country').size().reset_index(name='total_years')
missing_tfp = (missing_tfp
               .merge(total_years, on='country')
               .assign(missing_percentage = lambda x: (x.missing_count / x.total_years * 100)))
print("Países con mayor cantidad de valores faltantes en PTF:")
print("\nPaís\t\t\tValores Faltantes\tPorcentaje\tTotal Años")
print("-" * 70)
for _, row in missing_tfp.head(15).iterrows():
    print(f"{row['country']:<20} {int(row['missing_count']):<20} {row['missing_percentage']:,.1f}%\t\t{row['total_years']}")

6.  Agrupa los datos por país y calcula el promedio de las horas trabajadas anuales (avh) y de la educación promedio (hc) para el periodo 2000-2020.

In [ ]:
# 6.
data_2000_2020 = valid_data[(valid_data['year'] >= 2000) & (valid_data['year'] <= 2020)]
# Calcular promedios por país
country_averages = (data_2000_2020
                   .groupby('country')
                   .agg({
                       'avh': 'mean',  # promedio de horas trabajadas
                       'hc': 'mean',   # promedio de capital humano (educación)
                       'year': ['count', 'min', 'max']  # estadísticas de años disponibles
                   })
                   .round(2)  # redondear a 2 decimales
                   .reset_index())
# Renombrar columnas para claridad
country_averages.columns = ['country', 'avg_hours', 'avg_education', 'years_count', 'start_year', 'end_year']
# Ordenar por horas trabajadas descendente
country_averages_sorted = country_averages.sort_values('avg_hours', ascending=False)

print("\nPromedios por país (2000-2020):")
print("\nPaís                  Horas Trabajadas    Índice Educación    Periodo")
print("-" * 75)
for _, row in country_averages_sorted.iterrows():
    print(f"{row['country']:<20} {row['avg_hours']:>10,.1f}        {row['avg_education']:>8.2f}          {int(row['start_year'])}-{int(row['end_year'])}")

# Calcular algunas estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print("\nHoras trabajadas:")
print(country_averages['avg_hours'].describe().round(2))
print("\nÍndice de educación:")
print(country_averages['avg_education'].describe().round(2))

7.  Genere una variable que ordene a los países según el porcetaje de formación brutal de capital del 2019. Luego, estable diez grupos a manera de deciles. ¿En qué decil se ubica Péru? Vuelva a calcular usando la información de 1990. ¿El Perú se mantiene en el mismo décil o cambia?

In [ ]:
# 7. 
# Función para calcular deciles y posición de Perú
def calcular_deciles(data, año):
    # Filtrar datos del año específico
    data_año = data[data['year'] == año].copy()
    # Calcular el porcentaje de formación bruta de capital
    data_año['fbk_percent'] = data_año['csh_i'] * 100
    # Ordenar países y asignar deciles (1-10)
    data_año['decil'] = pd.qcut(data_año['fbk_percent'], q=10, labels=range(1,11))
    # Obtener posición de Perú
    peru_data = data_año[data_año['country'] == 'Peru']
    # Ordenar países por porcentaje para ver ranking completo
    ranking = data_año.sort_values('fbk_percent', ascending=False)
    return peru_data, ranking
# Calcular para 2019
peru_2019, ranking_2019 = calcular_deciles(valid_data, 2019)
# Calcular para 1990
peru_1990, ranking_1990 = calcular_deciles(valid_data, 1990)

print("Análisis de Formación Bruta de Capital:")
print("\nAño 2019:")
print(f"Perú se ubica en el decil: {peru_2019['decil'].iloc[0]}")
print(f"Porcentaje de FBK: {peru_2019['fbk_percent'].iloc[0]:.2f}%")

print("\nAño 1990:")
print(f"Perú se ubica en el decil: {peru_1990['decil'].iloc[0]}")
print(f"Porcentaje de FBK: {peru_1990['fbk_percent'].iloc[0]:.2f}%")

# Mostrar rankings completos
print("\nRanking 2019 (Top 10):")
print(ranking_2019[['country', 'fbk_percent', 'decil']].head(10))

print("\nRanking 1990 (Top 10):")
print(ranking_1990[['country', 'fbk_percent', 'decil']].head(10))

# Mostrar cambio en la posición relativa
cambio = peru_2019['decil'].iloc[0] - peru_1990['decil'].iloc[0]
print(f"\nCambio en la posición de Perú:")
if cambio == 0:
    print("Perú se mantiene en el mismo decil")
elif cambio > 0:
    print(f"Perú subió {abs(cambio)} deciles")
else:
    print(f"Perú bajó {abs(cambio)} deciles")

## **Transacciones Financieras**

Importa el conjunto de datos `base_financiera.csv`
    

8. Filtra todas las transacciones de tipo "Debit" en las que el monto sea mayor a 200, y calcula el monto total de esas transacciones. Además, determina cuántos clientes únicos realizaron esas transacciones.

In [ ]:
# Solución

9. Identifica las transacciones con valores nulos en la columna (`Amount`) . Rellena esos valores con la mediana de los montos por tipo de transacción (`Transaction_Type`) y luego calcula el monto total de transacciones por región (`region`) .

In [ ]:
# Solución

10. Busca transacciones duplicadas considerando las columnas (`Customer_ID`) y (`Date`). Elimina los duplicados manteniendo solo la primera ocurrencia. Luego, agrupa las transacciones restantes por (`Customer_ID`) y calcula el monto promedio y la cantidad de transacciones por cliente.

In [ ]:
# Solución